In [15]:
import pandas as pd
import vk
import numpy as np
import time
from datetime import datetime

1. Обработать из тхт файла в эксель айдишники
2. В таблице будет айди, статус обработки пользователя
3. После метод будет проходить по этим людям, собирать данные в датасет, в котором тоже будет статус
4. После по этой таблице будет проходить фссп майнер, если все норм, будет статус проставлять


Cвязать фссп таблицу и вк таблицу айдишниками

In [16]:
class ConfigAPI(object):
    __token = None
    api = None

    #Стартовый обработчик. Подключает TOKEN VK API к приложению
    def init(self):
        print('Вставьте ссылку в строку браузера для получения токена VK API. ')
        print('https://oauth.vk.com/authorize?client_id=7629255&display=page&scope=status,video,photos,friends,wall,notes,groups,docs&response_type=token&v=5.103&state=123456')
        print('После скопируйте ссылку из браузера и вставьте сюда')
        val = str(input())
        self.__get_token_by_link(val) #токен обрабатывается на ошибки в этом методе
        self.__vk_api_on() # подключение VK API к системе


    def __get_token_by_link(self, str):
        arr = str.split('=')
        if(len(arr) >= 2):
            arr2 = arr[1].split('&')
            token = arr2[0]
            print(token)
            self.__token = token
        else:
            print('ссылка повреждена, попробуйте еще раз!')
            print(' ')
            self.init()

    def __vk_api_on(self):
        token = self.__token
        try:
            if(token != None):
                session = vk.Session(token)
                self.api = vk.API(session, v=5.103)

                user = self.api.users.get(user_ids=1) #не работает...

                print('Токен подключен!')
                print(' ')
            else:
                print('Токен отсутствует, попробуйте еще раз!')
                print(' ')
                self.init()
        except Exception:
            print('Токен недействителен или поврежден, попробуйте еще раз')
            print(' ')
            self.init()

In [17]:
c = ConfigAPI()
c.init()

Вставьте ссылку в строку браузера для получения токена VK API. 
https://oauth.vk.com/authorize?client_id=7629255&display=page&scope=status,video,photos,friends,wall,notes,groups,docs&response_type=token&v=5.103&state=123456
После скопируйте ссылку из браузера и вставьте сюда
https://oauth.vk.com/blank.html#access_token=b2e4b8359d5f77f0d52f42312bbe1765980ad170c0cb51d4f22336830ab8cc2528fdbc14ff29dccddd254&expires_in=86400&user_id=599718705&state=123456
b2e4b8359d5f77f0d52f42312bbe1765980ad170c0cb51d4f22336830ab8cc2528fdbc14ff29dccddd254
Токен подключен!
 


In [21]:
# dataset = pd.DataFrame(columns = ['id', 'name', 'last_name', 'sex', 'bdate', 'city', 'home_town', 'has_mobile', 'contacts', 'site', 'education', 'universities', 'schools', 'status', 'followers_count', 'common_count', 'occupation', 'nickname', 'relatives', 'relation', 'connections', 'exports', 'activities', 'interests', 'music', 'movies', 'tv', 'books', 'games', 'about', 'quotes', 'can_see_audio', 'langs', 'people_main', 'life_main', 'smoking', 'alcohol', 'religion_id'])
# dataset.to_excel('vk_database.xlsx')
dataset = pd.read_excel('vk_database.xlsx')

In [8]:
df_ids = pd.read_excel('vk_ids.xlsx')
df_ids.head(1)

,id
0,485


In [26]:
# сhange i to last index in database
i = 54890
while i < len(df_ids):
    print('i: {}'.format(i))
    print('')
    people = c.api.users.get(user_ids = df_ids[i:i + 998]['id'], fields = ['sex', 'bdate', 'city', 'home_town', 'education', 'universities', 'schools', 'status', 'followers_count', 'common_count', 'occupation', 'nickname', 'relatives', 'relation', 'personal', 'connections', 'exports', 'activities', 'interests', 'music', 'movies', 'tv', 'books', 'games', 'about', 'quotes', 'can_see_audio'])
    for j in range(len(people)):
        print('j: {}'.format(j))
        if (df_ids['id'][i + j] == people[j]['id']):
            dataset.loc[i + j, 'id'] = people[j]['id']
            dataset.loc[i + j, 'name'] = people[j]['first_name']
            dataset.loc[i + j, 'last_name'] = people[j]['last_name']
            if (people[j].get('deactivated') == None):
                if (people[j]['is_closed'] == False):
                    if (people[j].get('sex') != None):
                        dataset.loc[i + j, 'sex'] = people[j]['sex']
                    if (people[j].get('bdate') != None):
                        dataset.loc[i + j, 'bdate'] = people[j]['bdate']
                    if (people[j].get('city') != None):
                        dataset.loc[i + j, 'city'] = people[j]['city']['title']
                    if (people[j].get('home_town') != None):
                        dataset.loc[i + j, 'home_town'] = people[j]['home_town']
                    if (people[j].get('education') != None):
                        dataset.loc[i + j, 'education'] = people[j]['education']
                    if (people[j].get('schools') != None):
                        dataset.loc[i + j, 'schools'] = len(people[j]['schools'])
                    if (people[j].get('status') != None):
                        dataset.loc[i + j, 'status'] = people[j]['status']
                    if (people[j].get('followers_count') != None):
                        dataset.loc[i + j, 'followers_count'] = people[j]['followers_count']
                    if (people[j].get('common_count') != None):
                        dataset.loc[i + j, 'common_count'] = people[j]['common_count']
                    if (people[j].get('occupation') != None):
                        dataset.loc[i + j, 'occupation'] = people[j]['occupation']['type']
                    if (people[j].get('nickname') != None):
                        dataset.loc[i + j, 'nickname'] = people[j]['nickname']
                    if (people[j].get('relatives') != None):
                        if (len(people[j]['relatives'])) > 0:
                            dataset.loc[i + j, 'relatives'] = len(people[j]['relatives'])
                    if (people[j].get('relation') != None):
                        dataset.loc[i + j, 'relation'] = people[j]['relation']
                    if (people[j].get('personal') != None):
                        if (people[j]['personal'].get('langs') != None):
                            dataset.loc[i + j, 'langs'] = len(people[j]['personal']['langs'])
                        if (people[j]['personal'].get('people_main') != None):
                            dataset.loc[i + j, 'people_main'] = people[j]['personal']['people_main']
                        if (people[j]['personal'].get('life_main') != None):
                            dataset.loc[i + j, 'life_main'] = people[j]['personal']['life_main']
                        if (people[j]['personal'].get('smoking') != None):
                            dataset.loc[i + j, 'smoking'] = people[j]['personal']['smoking']
                        if (people[j]['personal'].get('alcohol') != None):
                            dataset.loc[i + j, 'alcohol'] = people[j]['personal']['alcohol']
                        if (people[j]['personal'].get('religion_id') != None):
                            dataset.loc[i + j, 'religion_id'] = people[j]['personal']['religion_id']
                        
                    if (people[j].get('connections') != None):
                        dataset.iloc[i + j, 'connections'] = people[j]['connections']
                    if (people[j].get('exports') != None):
                        dataset.loc[i + j, 'exports'] = people[j]['exports']
                    if (people[j].get('activities') != None):
                        dataset.loc[i + j, 'activities'] = people[j]['activities']
                    if (people[j].get('interests') != None):
                        dataset.loc[i + j, 'interests'] = people[j]['interests']
                    if (people[j].get('music') != None):
                        dataset.loc[i + j, 'music'] = people[j]['music']
                    if (people[j].get('movies') != None):
                        dataset.loc[i + j, 'movies'] = people[j]['movies']
                    if (people[j].get('tv') != None):
                        dataset.loc[i + j, 'tv'] = people[j]['tv']
                    if (people[j].get('books') != None):
                        dataset.loc[i + j, 'books'] = people[j]['books']
                    if (people[j].get('games') != None):
                        dataset.loc[i + j, 'games'] = people[j]['games']
                    if (people[j].get('about') != None):
                        dataset.loc[i + j, 'about'] = people[j]['about']
                    if (people[j].get('quotes') != None):
                        dataset.loc[i + j, 'quotes'] = people[j]['quotes']
                    if (people[j].get('can_see_audio') != None):
                        dataset.loc[i + j, 'can_see_audio'] = people[j]['can_see_audio']
    print('saving vk db time: {}'.format(datetime.now()))
    dataset.to_excel('vk_database.xlsx')
    print('saved at: {}'.format(datetime.now()))
    i += 998
    if (i > len(dataset)):
        break
#         time.sleep(7)

i: 0

j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
j: 7
j: 8
j: 9
j: 10
j: 11
j: 12
j: 13
j: 14
j: 15
j: 16
j: 17
j: 18
j: 19
j: 20
j: 21
j: 22
j: 23
j: 24
j: 25
j: 26
j: 27
j: 28
j: 29
j: 30
j: 31
j: 32
j: 33
j: 34
j: 35
j: 36
j: 37
j: 38
j: 39
j: 40
j: 41
j: 42
j: 43
j: 44
j: 45
j: 46
j: 47
j: 48
j: 49
j: 50
j: 51
j: 52
j: 53
j: 54
j: 55
j: 56
j: 57
j: 58
j: 59
j: 60
j: 61
j: 62
j: 63
j: 64
j: 65
j: 66
j: 67
j: 68
j: 69
j: 70
j: 71
j: 72
j: 73
j: 74
j: 75
j: 76
j: 77
j: 78
j: 79
j: 80
j: 81
j: 82
j: 83
j: 84
j: 85
j: 86
j: 87
j: 88
j: 89
j: 90
j: 91
j: 92
j: 93
j: 94
j: 95
j: 96
j: 97
j: 98
j: 99
j: 100
j: 101
j: 102
j: 103
j: 104
j: 105
j: 106
j: 107
j: 108
j: 109
j: 110
j: 111
j: 112
j: 113
j: 114
j: 115
j: 116
j: 117
j: 118
j: 119
j: 120
j: 121
j: 122
j: 123
j: 124
j: 125
j: 126
j: 127
j: 128
j: 129
j: 130
j: 131
j: 132
j: 133
j: 134
j: 135
j: 136
j: 137
j: 138
j: 139
j: 140
j: 141
j: 142
j: 143
j: 144
j: 145
j: 146
j: 147
j: 148
j: 149
j: 150
j: 151
j: 152
j: 153
j: 154
j: 155
j: 156
j: 15

j: 208
j: 209
j: 210
j: 211
j: 212
j: 213
j: 214
j: 215
j: 216
j: 217
j: 218
j: 219
j: 220
j: 221
j: 222
j: 223
j: 224
j: 225
j: 226
j: 227
j: 228
j: 229
j: 230
j: 231
j: 232
j: 233
j: 234
j: 235
j: 236
j: 237
j: 238
j: 239
j: 240
j: 241
j: 242
j: 243
j: 244
j: 245
j: 246
j: 247
j: 248
j: 249
j: 250
j: 251
j: 252
j: 253
j: 254
j: 255
j: 256
j: 257
j: 258
j: 259
j: 260
j: 261
j: 262
j: 263
j: 264
j: 265
j: 266
j: 267
j: 268
j: 269
j: 270
j: 271
j: 272
j: 273
j: 274
j: 275
j: 276
j: 277
j: 278
j: 279
j: 280
j: 281
j: 282
j: 283
j: 284
j: 285
j: 286
j: 287
j: 288
j: 289
j: 290
j: 291
j: 292
j: 293
j: 294
j: 295
j: 296
j: 297
j: 298
j: 299
j: 300
j: 301
j: 302
j: 303
j: 304
j: 305
j: 306
j: 307
j: 308
j: 309
j: 310
j: 311
j: 312
j: 313
j: 314
j: 315
j: 316
j: 317
j: 318
j: 319
j: 320
j: 321
j: 322
j: 323
j: 324
j: 325
j: 326
j: 327
j: 328
j: 329
j: 330
j: 331
j: 332
j: 333
j: 334
j: 335
j: 336
j: 337
j: 338
j: 339
j: 340
j: 341
j: 342
j: 343
j: 344
j: 345
j: 346
j: 347
j: 348
j: 349
j: 350

j: 384
j: 385
j: 386
j: 387
j: 388
j: 389
j: 390
j: 391
j: 392
j: 393
j: 394
j: 395
j: 396
j: 397
j: 398
j: 399
j: 400
j: 401
j: 402
j: 403
j: 404
j: 405
j: 406
j: 407
j: 408
j: 409
j: 410
j: 411
j: 412
j: 413
j: 414
j: 415
j: 416
j: 417
j: 418
j: 419
j: 420
j: 421
j: 422
j: 423
j: 424
j: 425
j: 426
j: 427
j: 428
j: 429
j: 430
j: 431
j: 432
j: 433
j: 434
j: 435
j: 436
j: 437
j: 438
j: 439
j: 440
j: 441
j: 442
j: 443
j: 444
j: 445
j: 446
j: 447
j: 448
j: 449
j: 450
j: 451
j: 452
j: 453
j: 454
j: 455
j: 456
j: 457
j: 458
j: 459
j: 460
j: 461
j: 462
j: 463
j: 464
j: 465
j: 466
j: 467
j: 468
j: 469
j: 470
j: 471
j: 472
j: 473
j: 474
j: 475
j: 476
j: 477
j: 478
j: 479
j: 480
j: 481
j: 482
j: 483
j: 484
j: 485
j: 486
j: 487
j: 488
j: 489
j: 490
j: 491
j: 492
j: 493
j: 494
j: 495
j: 496
j: 497
j: 498
j: 499
j: 500
j: 501
j: 502
j: 503
j: 504
j: 505
j: 506
j: 507
j: 508
j: 509
j: 510
j: 511
j: 512
j: 513
j: 514
j: 515
j: 516
j: 517
j: 518
j: 519
j: 520
j: 521
j: 522
j: 523
j: 524
j: 525
j: 526

j: 577
j: 578
j: 579
j: 580
j: 581
j: 582
j: 583
j: 584
j: 585
j: 586
j: 587
j: 588
j: 589
j: 590
j: 591
j: 592
j: 593
j: 594
j: 595
j: 596
j: 597
j: 598
j: 599
j: 600
j: 601
j: 602
j: 603
j: 604
j: 605
j: 606
j: 607
j: 608
j: 609
j: 610
j: 611
j: 612
j: 613
j: 614
j: 615
j: 616
j: 617
j: 618
j: 619
j: 620
j: 621
j: 622
j: 623
j: 624
j: 625
j: 626
j: 627
j: 628
j: 629
j: 630
j: 631
j: 632
j: 633
j: 634
j: 635
j: 636
j: 637
j: 638
j: 639
j: 640
j: 641
j: 642
j: 643
j: 644
j: 645
j: 646
j: 647
j: 648
j: 649
j: 650
j: 651
j: 652
j: 653
j: 654
j: 655
j: 656
j: 657
j: 658
j: 659
j: 660
j: 661
j: 662
j: 663
j: 664
j: 665
j: 666
j: 667
j: 668
j: 669
j: 670
j: 671
j: 672
j: 673
j: 674
j: 675
j: 676
j: 677
j: 678
j: 679
j: 680
j: 681
j: 682
j: 683
j: 684
j: 685
j: 686
j: 687
j: 688
j: 689
j: 690
j: 691
j: 692
j: 693
j: 694
j: 695
j: 696
j: 697
j: 698
j: 699
j: 700
j: 701
j: 702
j: 703
j: 704
j: 705
j: 706
j: 707
j: 708
j: 709
j: 710
j: 711
j: 712
j: 713
j: 714
j: 715
j: 716
j: 717
j: 718
j: 719

j: 754
j: 755
j: 756
j: 757
j: 758
j: 759
j: 760
j: 761
j: 762
j: 763
j: 764
j: 765
j: 766
j: 767
j: 768
j: 769
j: 770
j: 771
j: 772
j: 773
j: 774
j: 775
j: 776
j: 777
j: 778
j: 779
j: 780
j: 781
j: 782
j: 783
j: 784
j: 785
j: 786
j: 787
j: 788
j: 789
j: 790
j: 791
j: 792
j: 793
j: 794
j: 795
j: 796
j: 797
j: 798
j: 799
j: 800
j: 801
j: 802
j: 803
j: 804
j: 805
j: 806
j: 807
j: 808
j: 809
j: 810
j: 811
j: 812
j: 813
j: 814
j: 815
j: 816
j: 817
j: 818
j: 819
j: 820
j: 821
j: 822
j: 823
j: 824
j: 825
j: 826
j: 827
j: 828
j: 829
j: 830
j: 831
j: 832
j: 833
j: 834
j: 835
j: 836
j: 837
j: 838
j: 839
j: 840
j: 841
j: 842
j: 843
j: 844
j: 845
j: 846
j: 847
j: 848
j: 849
j: 850
j: 851
j: 852
j: 853
j: 854
j: 855
j: 856
j: 857
j: 858
j: 859
j: 860
j: 861
j: 862
j: 863
j: 864
j: 865
j: 866
j: 867
j: 868
j: 869
j: 870
j: 871
j: 872
j: 873
j: 874
j: 875
j: 876
j: 877
j: 878
j: 879
j: 880
j: 881
j: 882
j: 883
j: 884
j: 885
j: 886
j: 887
j: 888
j: 889
j: 890
j: 891
j: 892
j: 893
j: 894
j: 895
j: 896

...

j: 721
j: 722
j: 723
j: 724
j: 725
j: 726
j: 727
j: 728
j: 729
j: 730
j: 731
j: 732
j: 733
j: 734
j: 735
j: 736
j: 737
j: 738
j: 739
j: 740
j: 741
j: 742
j: 743
j: 744
j: 745
j: 746
j: 747
j: 748
j: 749
j: 750
j: 751
j: 752
j: 753
j: 754
j: 755
j: 756
j: 757
j: 758
j: 759
j: 760
j: 761
j: 762
j: 763
j: 764
j: 765
j: 766
j: 767
j: 768
j: 769
j: 770
j: 771
j: 772
j: 773
j: 774
j: 775
j: 776
j: 777
j: 778
j: 779
j: 780
j: 781
j: 782
j: 783
j: 784
j: 785
j: 786
j: 787
j: 788
j: 789
j: 790
j: 791
j: 792
j: 793
j: 794
j: 795
j: 796
j: 797
j: 798
j: 799
j: 800
j: 801
j: 802
j: 803
j: 804
j: 805
j: 806
j: 807
j: 808
j: 809
j: 810
j: 811
j: 812
j: 813
j: 814
j: 815
j: 816
j: 817
j: 818
j: 819
j: 820
j: 821
j: 822
j: 823
j: 824
j: 825
j: 826
j: 827
j: 828
j: 829
j: 830
j: 831
j: 832
j: 833
j: 834
j: 835
j: 836
j: 837
j: 838
j: 839
j: 840
j: 841
j: 842
j: 843
j: 844
j: 845
j: 846
j: 847
j: 848
j: 849
j: 850
j: 851
j: 852
j: 853
j: 854
j: 855
j: 856
j: 857
j: 858
j: 859
j: 860
j: 861
j: 862
j: 863

j: 896
j: 897
j: 898
j: 899
j: 900
j: 901
j: 902
j: 903
j: 904
j: 905
j: 906
j: 907
j: 908
j: 909
j: 910
j: 911
j: 912
j: 913
j: 914
j: 915
j: 916
j: 917
j: 918
j: 919
j: 920
j: 921
j: 922
j: 923
j: 924
j: 925
j: 926
j: 927
j: 928
j: 929
j: 930
j: 931
j: 932
j: 933
j: 934
j: 935
j: 936
j: 937
j: 938
j: 939
j: 940
j: 941
j: 942
j: 943
j: 944
j: 945
j: 946
j: 947
j: 948
j: 949
j: 950
j: 951
j: 952
j: 953
j: 954
j: 955
j: 956
j: 957
j: 958
j: 959
j: 960
j: 961
j: 962
j: 963
j: 964
j: 965
j: 966
j: 967
j: 968
j: 969
j: 970
j: 971
j: 972
j: 973
j: 974
j: 975
j: 976
j: 977
j: 978
j: 979
j: 980
j: 981
j: 982
j: 983
j: 984
j: 985
j: 986
j: 987
j: 988
j: 989
j: 990
j: 991
j: 992
j: 993
j: 994
j: 995
j: 996
j: 997
saving vk db time: 2020-10-19 18:00:08.850879
saved at: 2020-10-19 18:01:33.456796
i: 51896

j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
j: 7
j: 8
j: 9
j: 10
j: 11
j: 12
j: 13
j: 14
j: 15
j: 16
j: 17
j: 18
j: 19
j: 20
j: 21
j: 22
j: 23
j: 24
j: 25
j: 26
j: 27
j: 28
j: 29
j: 30
j: 31
j: 32
j: 33

j: 72
j: 73
j: 74
j: 75
j: 76
j: 77
j: 78
j: 79
j: 80
j: 81
j: 82
j: 83
j: 84
j: 85
j: 86
j: 87
j: 88
j: 89
j: 90
j: 91
j: 92
j: 93
j: 94
j: 95
j: 96
j: 97
j: 98
j: 99
j: 100
j: 101
j: 102
j: 103
j: 104
j: 105
j: 106
j: 107
j: 108
j: 109
j: 110
j: 111
j: 112
j: 113
j: 114
j: 115
j: 116
j: 117
j: 118
j: 119
j: 120
j: 121
j: 122
j: 123
j: 124
j: 125
j: 126
j: 127
j: 128
j: 129
j: 130
j: 131
j: 132
j: 133
j: 134
j: 135
j: 136
j: 137
j: 138
j: 139
j: 140
j: 141
j: 142
j: 143
j: 144
j: 145
j: 146
j: 147
j: 148
j: 149
j: 150
j: 151
j: 152
j: 153
j: 154
j: 155
j: 156
j: 157
j: 158
j: 159
j: 160
j: 161
j: 162
j: 163
j: 164
j: 165
j: 166
j: 167
j: 168
j: 169
j: 170
j: 171
j: 172
j: 173
j: 174
j: 175
j: 176
j: 177
j: 178
j: 179
j: 180
j: 181
j: 182
j: 183
j: 184
j: 185
j: 186
j: 187
j: 188
j: 189
j: 190
j: 191
j: 192
j: 193
j: 194
j: 195
j: 196
j: 197
j: 198
j: 199
j: 200
j: 201
j: 202
j: 203
j: 204
j: 205
j: 206
j: 207
j: 208
j: 209
j: 210
j: 211
j: 212
j: 213
j: 214
j: 215
j: 216
j: 217
j: 218

j: 253
j: 254
j: 255
j: 256
j: 257
j: 258
j: 259
j: 260
j: 261
j: 262
j: 263
j: 264
j: 265
j: 266
j: 267
j: 268
j: 269
j: 270
j: 271
j: 272
j: 273
j: 274
j: 275
j: 276
j: 277
j: 278
j: 279
j: 280
j: 281
j: 282
j: 283
j: 284
j: 285
j: 286
j: 287
j: 288
j: 289
j: 290
j: 291
j: 292
j: 293
j: 294
j: 295
j: 296
j: 297
j: 298
j: 299
j: 300
j: 301
j: 302
j: 303
j: 304
j: 305
j: 306
j: 307
j: 308
j: 309
j: 310
j: 311
j: 312
j: 313
j: 314
j: 315
j: 316
j: 317
j: 318
j: 319
j: 320
j: 321
j: 322
j: 323
j: 324
j: 325
j: 326
j: 327
j: 328
j: 329
j: 330
j: 331
j: 332
j: 333
j: 334
j: 335
j: 336
j: 337
j: 338
j: 339
j: 340
j: 341
j: 342
j: 343
j: 344
j: 345
j: 346
j: 347
j: 348
j: 349
j: 350
j: 351
j: 352
j: 353
j: 354
j: 355
j: 356
j: 357
j: 358
j: 359
j: 360
j: 361
j: 362
j: 363
j: 364
j: 365
j: 366
j: 367
j: 368
j: 369
j: 370
j: 371
j: 372
j: 373
j: 374
j: 375
j: 376
j: 377
j: 378
j: 379
j: 380
j: 381
j: 382
j: 383
j: 384
j: 385
j: 386
j: 387
j: 388
j: 389
j: 390
j: 391
j: 392
j: 393
j: 394
j: 395

KeyboardInterrupt: 

In [1]:
# dataset = pd.read_excel('vk_database.xlsx')

with open("vk_ids_kzn.txt", "r") as a_file:
    for line in a_file:
        stripped_line = line.strip()
        length = len(dataset)
        dataset.loc[length, 'id'] = stripped_line
        print(length)

dataset.to_excel('vk_database.xlsx') 
# сохранять каждый раз, после удалить, которые обработаны, запустить дальше